In [1]:
import imprint as ip
ip.setup_nb()

[worker_id=None] 2023-03-16 11:20:40,551 - imprint - DEBUG 
Enabling 64-bit floats in JAX.


In [2]:
from confirm.cloud.coiled_backend import CoiledBackend
import confirm.adagrid as ada
from imprint.models.ztest import ZTest1D

backend = CoiledBackend()

[worker_id=None] 2023-03-16 11:20:41,974 - confirm.cloud.coiled_backend - DEBUG 
# THIS IS AUTOGENERATED BY build_coiled_env.py  DO NOT MODIFY.
# THIS IS AUTOGENERATED BY build_coiled_env.py  DO NOT MODIFY.
# THIS IS AUTOGENERATED BY build_coiled_env.py  DO NOT MODIFY.
# THIS IS AUTOGENERATED BY build_coiled_env.py  DO NOT MODIFY.
name: confirm
channels:
  - conda-forge
  - nvidia
dependencies:
  - python=3.10
  - nvidia:cuda-toolkit=11.8
  - conda-forge:cudnn=8.4
  - pip
  - pip:
    - absl-py==1.4.0
    - cloudpickle==2.0.0
    - colorama==0.4.6
    - decorator==5.1.1
    - dm-tree==0.1.8
    - duckdb==0.7.1
    - gast==0.5.3
    - mpmath==1.3.0
    - multipledispatch==0.6.0
    - numpy==1.24.2
    - numpyro==0.11.0
    - opt-einsum==3.3.0
    - pandas==1.5.3
    - python-dateutil==2.8.2
    - pytz==2022.7.1
    - scipy==1.10.1
    - six==1.16.0
    - sympy==1.11.1
    - synchronicity==0.2.14
    - tensorflow-probability==0.19.0
    - tqdm==4.65.0
    - --find-links https://storage.g

Output()

/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/distributed/client.py:1361: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | Client | Scheduler | Workers |
+---------+--------+-----------+---------+
| lz4     | 4.3.2  | None      | None    |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [6]:
g = ip.cartesian_grid(theta_min=[-1], theta_max=[1], null_hypos=[ip.hypo("x0 < 0")])
db = ada.ada_calibrate(
    ZTest1D, g=g, nB=5, tile_batch_size=1, prod=True, n_zones=1, backend=backend
)

[worker_id=None] 2023-03-16 11:23:03,413 - imprint.grid - DEBUG 
_gen_short_uuids(n=2, worker_id=1, t=1678980183, n_bits=18, worker_bits=18) = [4698175183616475136 4698175183616475137, ...]:
[worker_id=1] 2023-03-16 11:23:04,905 - confirm.adagrid.init - DEBUG 
Initialized database with 1 tiles and 1 null hypos. The tiles are split between 1 zones with packet_size=1024.
[worker_id=1] 2023-03-16 11:23:04,906 - confirm.adagrid.init - INFO 
Config (minus system info): 
{'model_seed': 0, 'alpha': 0.025, 'init_K': 8192, 'n_K_double': 4, 'bootstrap_seed': 0, 'nB': 5, 'tile_batch_size': 1, 'grid_target': 0.001, 'bias_target': 0.001, 'std_target': 0.002, 'calibration_min_idx': 40, 'n_steps': 100, 'timeout': 43200, 'step_size': 1024, 'packet_size': 1024, 'coordinate_every': 1, 'n_zones': 1, 'prod': True, 'job_name': None, 'model_name': 'ZTest1D', 'model_kwargs_json': '{}', 'worker_id': 1, 'jax_platform': 'cpu', 'git_hash': '56b2103afabb6faa5a951f564df6831a8a991070', 'platform': 'macOS-13.2.1-arm

In [7]:
db

DuckDBTiles(con=<duckdb.DuckDBPyConnection object at 0x167813230>, _tiles_columns_cache=Index(['id', 'active', 'parent_id', 'theta0', 'radii0', 'null_truth0', 'K',
       'coordination_id', 'step_id', 'zone_id', 'packet_id', 'creator_id',
       'creation_time'],
      dtype='object'), _results_columns_cache=Index(['id', 'active', 'parent_id', 'theta0', 'radii0', 'null_truth0', 'K',
       'coordination_id', 'step_id', 'zone_id', 'packet_id', 'creator_id',
       'creation_time', 'processor_id', 'processing_time', 'eligible',
       'grid_cost', 'impossible', 'orderer', 'alpha0', 'idx', 'twb_max_lams',
       'twb_mean_lams', 'twb_min_lams', 'lams', 'B_lams0', 'B_lams1',
       'B_lams2', 'B_lams3', 'B_lams4', 'twb_lams0', 'twb_lams1', 'twb_lams2',
       'twb_lams3', 'twb_lams4'],
      dtype='object'), _d=None, is_distributed=False, supports_threads=False)